In [1]:
# %%capture

# !pip install pyspellchecker==0.7.0
# !pip install -q transformers

# !pip install sentencepiece
# !pip install git+https://github.com/csebuetnlp/normalizer

# ! pip install bangla==0.0.2
# # !pip install num2words
!nvidia-smi

Tue Nov  1 12:36:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    Off  | 00000000:65:00.0 Off |                  Off |
| 30%   47C    P2    74W / 300W |   3551MiB / 49140MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import re
# from num2words import num2words
import os
from tqdm.auto import tqdm
tqdm.pandas()

import transformers

import torch
import bangla
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,pipeline
from normalizer import normalize # pip install git+https://github.com/csebuetnlp/normalizer

import warnings
warnings.filterwarnings("ignore")

# from pandarallel import pandarallel
# pandarallel.initialize(progress_bar=True,nb_workers=8)

os.environ["TOKENIZERS_PARALLELISM"] = "true"

print(torch.__version__)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/home/ansary/anaconda3/envs/mobassir/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.12.1


In [3]:
!pwd

/home/ansary/Shabab/tafsir_jalalayn


In [4]:
!ls

'en_Tafsir al-Jalalayn.csv'   tafsir-al-jalalayn-en-to-bn-translator.ipynb


In [5]:
tafsir_en = pd.read_csv('/home/ansary/Shabab/tafsir_jalalayn/en_Tafsir al-Jalalayn.csv')
tafsir_en.head()

,ayat,তাইসিরুল,en_tafsir_jalalayn
0,"surah 1, ayat 1",(আরম্ভ করছি) পরম করুণাময় অসীম দয়াময় আল্লাহর না...,In the Name of God the Compassionate the Merciful
1,"surah 1, ayat 2",যাবতীয় প্রশংসা জগৎসমূহের প্রতিপালক আল্লাহরই জন...,In the Name of God the name of a thing is that...
2,"surah 1, ayat 3",যিনি পরম করুণাময় অতি দয়ালু।,The Compassionate the Merciful that is to say ...
3,"surah 1, ayat 4",যিনি প্রতিফল দিবসের মালিক।,Master of the Day of Judgement that is the day...
4,"surah 1, ayat 5",আমরা কেবল তোমারই ‘ইবাদাত করি এবং কেবলমাত্র তোম...,You alone we worship and You alone we ask for ...


In [6]:

print(torch.__version__)
transformers.__version__

1.12.1


'4.22.1'

# single sample inference test

thanks to banglanmt and  meta's nllb-200

In [7]:
torch_device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(torch_device)
model1 = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5_nmt_en_bn").to(torch_device)
tokenizer1 = AutoTokenizer.from_pretrained("csebuetnlp/banglat5_nmt_en_bn",use_fast=True)



def banglanmt_translate_en_bn(input_sentence):
    input_ids = tokenizer1(normalize(input_sentence), return_tensors="pt").input_ids
    input_ids = input_ids.to(torch_device)
    generated_tokens = model1.generate(input_ids)#max_length=1536
    decoded_tokens = tokenizer1.batch_decode(generated_tokens)[0]
    decoded_tokens=decoded_tokens.replace("<pad>","").replace("</s>","")
    sen=decoded_tokens.split()
    words=[w for w in sen if w.strip()]
    sen="".join(words)
    return decoded_tokens

# !pip install -q transformers

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-1.3B")
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-1.3B")


translation_pipeline = pipeline('translation', 
                                model=model, 
                                tokenizer=tokenizer, 
                                src_lang="eng_latn",  #arb_Arab,eng_latn
                                tgt_lang='ben_Beng',
                                device = torch_device,
                                max_length = 1024)

def translate_en_bn(input_sentence):
    sentences = re.split(r'[।!.,?]', input_sentence)
    bn = []
    for i in range(len(sentences)):
        result = translation_pipeline(sentences[i])
        bn.append(result[0]['translation_text'])
    bn = " ".join(bn)
    bn = re.sub(' ্ ','',bn)
    bn = re.sub("\\'","",bn)#replace \'
    bn = re.sub('<unk>','',bn)
    return bn

en_text = "Which was revealed in Makkah"
print("banglanmt -> \n",banglanmt_translate_en_bn(en_text))
print("\nnllb-200 -> \n",)
translate_en_bn(en_text)

cuda:0
banglanmt -> 
  যা মক্কায় অবতীর্ণ হয়েছিল

nllb-200 -> 



'যা মক্কায় অবতীর্ণ হয়েছে'

In [8]:
%%time

en_text = '''In the Name of Allah, the Most Gracious, the Most Merciful. <ar> ﴿الْحَمْدُ للَّهِ رَبِّ الْعَـلَمِينَ﴾ </ar>
(Allah, the Exalted, said, `I have divided the prayer (Al-Fatihah) into two halves between 
Myself and My servant, and My servant shall have what he asks for.
If he says,' <ar> بِسْمِ اللَّهِ الرَّحْمَـنِ الرَّحِيمِ </ar> Allah says, `My servant has praised Me.' '''

def EN_AR_to_BN_AR_Translator(en_text):
    '''
    translates multilingual english-arabic code mixed text into 
    multilingual bengali-arabic code mixed text
    ''' 

    sentenceEnders = re.compile('[.,!?]')
    sentences = sentenceEnders.split(en_text)
    main_list = []
    for i in range(len(sentences)):
        list_str = sentences[i].split('<ar>')
        if(len(list_str) == 1):
            main_list.append(list_str[0])
        else:
            for j in range(len(list_str)):
                if('</ar>' in list_str[j]):
                    list_str1 = list_str[j].split('</ar>')
                    main_list.append("<ar>"+list_str1[0]+"</ar>")
                    main_list.append(list_str1[1])
                else:
                    main_list.append(list_str[j])

    while(" " in main_list):
        main_list.remove(" ")
        
    for idx in range(len(main_list)):
        if('<ar>' not in main_list[idx] or '</ar>' not in main_list[idx]):
            
            output_sentence = []
            for word in main_list[idx].split():
                output_sentence.append(word)
     
            main_list[idx] = ' '.join(output_sentence)
            #numerizer
            main_list[idx] = bangla.convert_english_digit_to_bangla_digit(main_list[idx])
            # multilingual english-arabic to multilingual bengali-arabic
            try:
                main_list[idx] = banglanmt_translate_en_bn(main_list[idx])
            except:
                main_list[idx] = translate_en_bn(main_list[idx])
                print("banglanmt failed for -> ",main_list[idx])
    bn_mlt = " ".join(main_list)
    bn_mlt = re.sub(' ্ ','',bn_mlt)
    bn_mlt = re.sub("\\'","",bn_mlt)#replace \'
    bn_mlt = re.sub('<unk>','',bn_mlt)
    return bn_mlt
        
EN_AR_to_BN_AR_Translator(en_text)

CPU times: user 1.98 s, sys: 8.59 ms, total: 1.99 s
Wall time: 1.99 s


' আল্লাহর নামে  পরাৎপর  পরম করুণাময় <ar> ﴿الْحَمْدُ للَّهِ رَبِّ الْعَـلَمِينَ﴾ </ar>  (আল্লাহ  মহামহিম  কথিত  আমি আমার ও আমার দাসের মধ্যে প্রার্থনাকে (আল-ফাতিহা) দুই ভাগে ভাগ করেছি  এবং আমার বান্দা যা চায় তা পাবে  যদি সে বলে   <ar> بِسْمِ اللَّهِ الرَّحْمَـنِ الرَّحِيمِ </ar>  আল্লাহ বলেন  আমার দাস আমার প্রশংসা করেছে  '

quality of translation looks much better!

In [9]:
%%time
tafsir_en["Tafsir_bn"]=tafsir_en.en_tafsir_jalalayn.progress_apply(lambda tafsir_eng: EN_AR_to_BN_AR_Translator(tafsir_eng))


100%|█████████████████████████████████████| 6236/6236 [2:44:54<00:00,  1.59s/it]

CPU times: user 2h 44min 42s, sys: 12.3 s, total: 2h 44min 54s
Wall time: 2h 44min 54s


In [10]:
tafsir_en["Tafsir_bn"][0]

' করুণাময় দয়ালু আল্লাহর নামে'

In [11]:

tafsir_en.to_csv('/home/ansary/Shabab/tafsir_jalalayn/nllb-200_Multilingual_bn_ar_tafsir_al_jalalayn.csv',index = False)

In [12]:
tafsir_en.head()

,ayat,তাইসিরুল,en_tafsir_jalalayn,Tafsir_bn
0,"surah 1, ayat 1",(আরম্ভ করছি) পরম করুণাময় অসীম দয়াময় আল্লাহর না...,In the Name of God the Compassionate the Merciful,করুণাময় দয়ালু আল্লাহর নামে
1,"surah 1, ayat 2",যাবতীয় প্রশংসা জগৎসমূহের প্রতিপালক আল্লাহরই জন...,In the Name of God the name of a thing is that...,"ঈশ্বরের নামে কোন কিছুর নাম হয়, যার দ্বারা তা..."
2,"surah 1, ayat 3",যিনি পরম করুণাময় অতি দয়ালু।,The Compassionate the Merciful that is to say ...,করুণাময় করুণাময় বলতে বোঝায় যিনি দয়ার অধিক...
3,"surah 1, ayat 4",যিনি প্রতিফল দিবসের মালিক।,Master of the Day of Judgement that is the day...,কেয়ামতের দিন যা কেয়ামতের দিন বিচারের দিন স...
4,"surah 1, ayat 5",আমরা কেবল তোমারই ‘ইবাদাত করি এবং কেবলমাত্র তোম...,You alone we worship and You alone we ask for ...,আপনি একা আমরা উপাসনা করি এবং আপনি একা আমরা সা...
